## AIDevs3 tasks from newest to oldest, work in progress...

# S01E01

In [6]:
# Standard library imports
import requests
import re

# Third-party imports
from utilities.common import OpenAIClient
from utilities.config import S01E01_TASK_URL, S01E01_TASK_USERNAME, S01E01_TASK_PASSWORD

# Configuration 
url, username, password = S01E01_TASK_URL, S01E01_TASK_USERNAME, S01E01_TASK_PASSWORD

# Initialize OpenAI client
client_openai = OpenAIClient()

def get_current_question():
    """Fetches the current question from the specified URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        match = re.search(r'<p id="human-question">(.*?)</p>', response.text)
        return match.group(1).strip() if match else None
    except requests.RequestException as err:
        print(f"Error: {err}")
        return None

# Fetch question and get AI-generated answer
question = get_current_question()
if question:
    answer = client_openai.get_completion(
        prompt=f"Answer just numbers: {question}", temperature=0.1
    )
    print(question, " Answer:", answer)

    # Submit login data and retrieve flag if available
    response = requests.post(
        url,
        data={'username': username, 'password': password, 'answer': answer}
    )
    if response.status_code == 200:
        match = re.search(r"\{FLG:(.*?)\}", response.text)
        print("{FLG:} found:", match.group(1).strip() if match else "No flag found.")
    else:
        print(f"Login failed. Status: {response.status_code}")
else:
    print("Failed to retrieve the question.")

Question:<br />Rok lądowania na Księżycu?  Answer: 1969
{FLG:} found: FIRMWARE


# S00E01
At the address below, there are two strings (note! They change from time to time, so don't hardcode them in your code!).
Your task is to retrieve them and send them back as an array of strings to the API endpoint:

In [1]:
# Import necessary modules
from utilities.common import AIDevsClient
from utilities.config import S00E01_DATA_URL

# Configuration
task_name = "POLIGON"
data_url = S00E01_DATA_URL

# Initialize the client
client_aidevs = AIDevsClient()

# Step 1: Retrieve data using the client method
data_list = client_aidevs.fetch_data(data_url)

# Step 2: Submit the answer
if data_list:
    submit_response = client_aidevs.submit_answer(task_name, data_list)
    print("Submit response:", submit_response)

Submit response: {'code': 0, 'message': 'Super. Wszystko OK!'}
